In [1]:
%pip list | grep catboost
%pip install catboost --upgrade
%pip install pandas
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split

data_path = "Tabular Dataset.csv"
pokemon_df = pd.read_csv(data_path)

# Drop unnecessary columns definitively
columns_to_drop = ['National\nDex', 'Pokemon\nName']
pokemon_df = pokemon_df.drop(columns=columns_to_drop, errors='ignore')

# Check if columns were correctly dropped
print("Remaining columns in dataset:", pokemon_df.columns)

# Define `X` and `y` based on available columns
type_columns = ['Type I', 'Type II']
X = pokemon_df.drop(columns=type_columns, errors='ignore')
y = pokemon_df[type_columns].fillna('Unknown')

# Combine types for the target variable
y['Combined'] = y.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
y_combined = y['Combined']

# Filter classes with low occurrence
class_counts = y_combined.value_counts()
valid_classes = class_counts[class_counts >= 2].index
valid_indices = y_combined.isin(valid_classes)

X_valid = X[valid_indices]
y_valid = y_combined[valid_indices]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_valid, y_valid, test_size=0.2, random_state=42, stratify=y_valid)

# Filter test data to include only classes in the training set
train_classes = set(y_train)
test_indices = y_test.apply(lambda x: x in train_classes)
X_test = X_test[test_indices]
y_test = y_test[test_indices]

# Identify and set relevant categorical columns
categorical_cols = ['Ability I', 'Ability II', 'Hidden Ability', 'Pokedex Color', 'Egg Group I', 'Egg Group II']
categorical_cols = [col for col in categorical_cols if col in X_train.columns]

# Convert categorical columns to strings
for col in categorical_cols:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    X_valid[col] = X_valid[col].astype(str)

# Create Pool objects
train_pool = Pool(X_train, y_train, cat_features=categorical_cols)
test_pool = Pool(X_test, y_test, cat_features=categorical_cols)
valid_pool = Pool(X_valid, y_valid, cat_features=categorical_cols)

# Initialize and train model
model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='MultiClass', eval_metric='MultiClass')

# Perform cross-validation
cv_params = model.get_params()
cv_data = cv(valid_pool, cv_params, fold_count=10, plot=True)

# Fit the model
model.fit(train_pool, eval_set=valid_pool, verbose=100)

# Make predictions
predictions = model.predict_proba(test_pool)
print(predictions)



Remaining columns in dataset after dropping 'National Dex' and 'Pokemon Name': Index(['National\r\nDex', 'Pokemon\r\nName', 'HP', 'Attack', 'Defense',
       'Special\r\nAttack', 'Special\r\nDefense', 'Speed', 'Total',
       'Ability I', 'Ability II', 'Hidden Ability', 'Weight (kg)',
       'Height (m)', 'Pokedex Color', 'Egg Group I', 'Egg Group II', 'Type I',
       'Type II'],
      dtype='object')
Data types of X_train columns before Pool: National\r\nDex       float64
Pokemon\r\nName        object
HP                      int64
Attack                  int64
Defense                 int64
Special\r\nAttack       int64
Special\r\nDefense      int64
Speed                   int64
Total                   int64
Ability I              object
Ability II             object
Hidden Ability         object
Weight (kg)           float64
Height (m)            float64
Pokedex Color          object
Egg Group I            object
Egg Group II           object
dtype: object


C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna('Unknown', inplace=True)
C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Combined'] = y.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
C:\Users\maxfe\AppData\Local\Temp\ipykernel_110936\1199800706.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="Pelipper": Cannot convert 'b'Pelipper'' to float